In [12]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.cluster import KMeans

import tensorflow as tf
from tensorflow import keras
# from tensorflow.keras import layers

# import tensorflow_docs.modeling

In [13]:
url = 'https://raw.githubusercontent.com/skjdfhkskjds/machine-learning-projects/main/heart.csv'
df = pd.read_csv(url)
df.tail()
df.isnull().sum().sort_values(ascending=False)/df.shape[0]

Age               0.0
Sex               0.0
ChestPainType     0.0
RestingBP         0.0
Cholesterol       0.0
FastingBS         0.0
RestingECG        0.0
MaxHR             0.0
ExerciseAngina    0.0
Oldpeak           0.0
ST_Slope          0.0
HeartDisease      0.0
dtype: float64

In [14]:
features = ['Age','Sex','ChestPainType','RestingBP','Cholesterol','FastingBS','RestingECG','MaxHR','ExerciseAngina','Oldpeak','ST_Slope']
numerical_features = ['Age','RestingBP','Cholesterol','FastingBS','MaxHR','Oldpeak']
categorical_features = ['Sex','ChestPainType','RestingECG','ExerciseAngina','ST_Slope']

#Instantiate SimpleImputer 
si=SimpleImputer(missing_values = np.nan, strategy="median")
si.fit(df[numerical_features])
  
#Filling missing data with median
df[numerical_features] = si.transform(df[numerical_features])

for category in categorical_features:
  df[category] = pd.factorize(df[category])[0]

#Adds a new feature called type of patient based on the kmeans clustering of the dataset
kmeans = KMeans(n_clusters=2)
kmeans.fit(df[features])
df['typeOfPatient'] = kmeans.predict(df[features])

In [15]:
#splits the dataset into 80:20 training and testing data
y = df.pop('HeartDisease')
dftrain, dftest, y_train, y_test = train_test_split(df, y, test_size=0.2, random_state=0)

In [16]:
from sklearn.preprocessing import StandardScaler
# Scaling numeric features using sklearn StandardScalar
sc = StandardScaler()
dftrain[numerical_features] = sc.fit_transform(dftrain[numerical_features])
dftest[numerical_features] = sc.transform(dftest[numerical_features])

model = keras.Sequential([
    keras.layers.Flatten(),
    keras.layers.Dense(16, activation=tf.nn.relu),
    keras.layers.Dense(16, activation=tf.nn.relu),
    keras.layers.Dense(1, activation=tf.nn.sigmoid),
])

C:\Users\colin\anaconda3\lib\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)
C:\Users\colin\anaconda3\lib\site-packages\pandas\core\frame.py:3678: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = igetitem(value, i)


In [17]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [18]:
model.fit(dftrain, y_train, epochs=60, batch_size=1)

Epoch 1/60
734/734 [==============================] - 1s 714us/step - loss: 0.5136 - accuracy: 0.7616
Epoch 2/60
734/734 [==============================] - 1s 690us/step - loss: 0.3876 - accuracy: 0.8501
Epoch 3/60
734/734 [==============================] - 1s 729us/step - loss: 0.3567 - accuracy: 0.8638
Epoch 4/60
734/734 [==============================] - 1s 720us/step - loss: 0.3404 - accuracy: 0.8733
Epoch 5/60
734/734 [==============================] - 1s 723us/step - loss: 0.3245 - accuracy: 0.8801
Epoch 6/60
734/734 [==============================] - 1s 700us/step - loss: 0.3179 - accuracy: 0.8706
Epoch 7/60
734/734 [==============================] - 1s 712us/step - loss: 0.3075 - accuracy: 0.8787
Epoch 8/60
734/734 [==============================] - 1s 688us/step - loss: 0.2948 - accuracy: 0.8883
Epoch 9/60
734/734 [==============================] - 1s 707us/step - loss: 0.2913 - accuracy: 0.8828
Epoch 10/60
734/734 [==============================] - 1s 720us/step - loss: 0.283

In [19]:
test_loss, test_acc = model.evaluate(dftest, y_test)
print('Test accuracy:', test_acc)

6/6 [==============================] - 0s 600us/step - loss: 0.7065 - accuracy: 0.8424
Test accuracy: 0.842391312122345


In [20]:
model.save('heart_disease_prediction.h5')